In [58]:
import numpy as np
import scipy
np.set_printoptions(precision=1)

In [3]:
def get_normalized_Hamiltonian(U):
    '''
    Compute H, where U = exp(iHt), where H is Hermitian and has trace norm of 1.
    :param U:
    :return:
    '''

    assert len(U.shape) == 2 and U.shape[0] == U.shape[1], "The input matrix is not square."
    assert np.linalg.norm(U @ U.conj().T - np.eye(U.shape[0])) < 10e-6, "The input matrix is not unitary."

    iHt = scipy.linalg.logm(U)
    t = np.linalg.norm(iHt,'nuc')

    if t != 0:
        H = -1j * iHt / t
        return H,t
    else:
        return iHt,0

In [4]:
import qiskit.circuit.library as library

SINGLE_QUBIT_DETERMINISTIC_GATES = ['h', 'x', 'y', 'z']
SINGLE_QUBIT_VARIATIONAL_GATES = ['rx', 'ry', 'rz']
TWO_QUBIT_DETERMINISTIC_GATES = ['cx', 'cy', 'cz']
TWO_QUBIT_VARIATIONAL_GATES = ['crx', 'cry', 'crz', 'rxx', 'ryy', 'rzz']
ADMISSIBLE_GATES = SINGLE_QUBIT_DETERMINISTIC_GATES + SINGLE_QUBIT_VARIATIONAL_GATES + TWO_QUBIT_DETERMINISTIC_GATES + TWO_QUBIT_VARIATIONAL_GATES

UNITARY = {'h': library.HGate, 'x': library.XGate, 'y': library.YGate, 'z': library.ZGate,
           'rx': library.RXGate, 'ry': library.RYGate, 'rz': library.RZGate,
           'cx': library.CXGate, 'cy':library.CYGate, 'cz':library.CZGate,
           'crx':library.CRXGate, 'cry':library.CRYGate, 'crz':library.CRZGate,
           'rxx':library.RXXGate, 'ryy':library.RYYGate, 'rzz':library.RZZGate}

In [104]:
from qiskit import QuantumCircuit
import qiskit.quantum_info as qi

qc1 = QuantumCircuit(4)
qc1.append(UNITARY['cx'](),[0,1])
print(qc1.draw())

op1 = qi.Operator(qc1)
matrix1 = op1.data
print(matrix1)

qc2 = QuantumCircuit(4)
qc2.append(UNITARY['cx'](), [3,2])
print(qc2.draw())

op2 = qi.Operator(qc2)
matrix2 = op2.data
print(matrix2)

          
q_0: ──■──
     ┌─┴─┐
q_1: ┤ X ├
     └───┘
q_2: ─────
          
q_3: ─────
          
[[1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
  0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
  0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
  0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
  0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
  0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j
  0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j
  0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
  0.+0.j 0

In [105]:
H1, t1 = get_normalized_Hamiltonian(matrix1)
H2, t2 = get_normalized_Hamiltonian(matrix2)

rec1 = scipy.linalg.expm(1j * H1 * t1)
rec2 = scipy.linalg.expm(1j * H2 * t2)
print(np.allclose(matrix1, rec1), np.allclose(matrix2, rec2))

print(np.linalg.norm(H1 - H2, ord='nuc'))


True True
1.4999999999999993
